In [1]:
import pandas as pd

# CSV 파일 경로 설정
file_path = './tour_distances_matrix.csv'
DURATION_PATH = "./residence_time.csv"

# CSV 파일 로드
duration_matrix_df = pd.read_csv(file_path, index_col=0)

driving_time_matrix = duration_matrix_df.values
travel_point_name = duration_matrix_df.columns.to_list()

duration_matrix_df = pd.read_csv(DURATION_PATH, index_col=0)
duration_time_list = duration_matrix_df["Avg_Stay_Duration"].values



In [2]:
import numpy as np
import random
from deap import base, creator, tools, algorithms
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

random.seed(64)
algorithms.random.seed(64)

total_day = 7

# 유전 알고리즘 설정
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# 각 관광지를 무작위 군집에 할당하는 함수
def create_individual():
    return [random.randint(0, total_day-1) for _ in range(len(driving_time_matrix) - 1)]

toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# OR-Tools를 사용하여 군집 내 관광지 순회 시간 계산
def calculate_cluster_time(cluster):
    # 숙소(인덱스 0)를 시작점과 종료점으로 추가
    cluster_with_base = [0] + cluster + [0]

    if len(cluster_with_base) <= 2:
        return 0  # 군집 내 관광지가 없으면 이동 시간은 0

    manager = pywrapcp.RoutingIndexManager(len(cluster_with_base), 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    # 거리 콜백 정의
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return driving_time_matrix[cluster_with_base[from_node], cluster_with_base[to_node]]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)

    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        # 순회 시간 계산
        return solution.ObjectiveValue()
    else:
        return 0


# 평가 함수
def evalTour(individual):
    # 군집별로 관광지 인덱스를 분류
    clusters = [[] for _ in range(total_day)]
    for idx, cluster_id in enumerate(individual):
        clusters[cluster_id].append(idx + 1)  # 관광지 인덱스는 1부터 시작
    # 각 군집의 순회 시간 계산
    times = [calculate_cluster_time(cluster) for cluster in clusters]  # 숙소 포함하여 계산
    total_travel_time = np.sum(times)
    # 순회 시간의 표준 편차 반환
    if len(times) > 1:
        travel_time_std = np.std(times)
    else:
        travel_time_std = 0  # 단일 군집의 경우 표준 편차는 0

    alpha = 0.15
    fitness =  alpha * total_travel_time + (1-alpha) * travel_time_std

    return (fitness,)


toolbox.register("evaluate", evalTour)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# 유전 알고리즘 실행
population_size = 100
num_generations = 100

pop = toolbox.population(n=population_size)
hof = tools.HallOfFame(1, similar=np.array_equal)

result = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=num_generations, halloffame=hof, verbose=True)

# 최적 개체 출력
best_individual = hof.items[0]
print("Best Individual = ", best_individual)
print("Fitness = ", evalTour(best_individual))

gen	nevals
0  	100   
1  	80    
2  	71    
3  	75    
4  	76    
5  	78    
6  	76    
7  	77    
8  	69    
9  	60    
10 	75    
11 	79    
12 	80    
13 	85    
14 	83    
15 	81    
16 	77    
17 	75    
18 	77    
19 	72    
20 	83    
21 	84    
22 	72    
23 	79    
24 	64    
25 	78    
26 	71    
27 	74    
28 	75    
29 	79    
30 	75    
31 	68    
32 	77    
33 	74    
34 	81    
35 	72    
36 	77    
37 	79    
38 	81    
39 	76    
40 	82    
41 	82    
42 	73    
43 	81    
44 	80    
45 	77    
46 	81    
47 	77    
48 	82    
49 	66    
50 	83    
51 	74    
52 	77    
53 	72    
54 	72    
55 	80    
56 	74    
57 	75    
58 	73    
59 	68    
60 	76    
61 	81    
62 	76    
63 	73    
64 	74    
65 	78    
66 	86    
67 	79    
68 	71    
69 	81    
70 	74    
71 	70    
72 	78    
73 	74    
74 	74    
75 	81    
76 	70    
77 	82    
78 	74    
79 	77    
80 	74    
81 	78    
82 	73    
83 	71    
84 	84    
85 	84    
86 	78    
87 	72    
88 	76    
89 	66    

In [5]:
def solve_tsp_for_cluster(cluster):
    # 숙소를 포함하여 TSP 문제 설정
    if len(cluster) == 0:
        return [0]  # 군집 내 관광지가 없으면 숙소만 반환
    cluster_with_base = [0] + cluster # 숙소를 포함
    manager = pywrapcp.RoutingIndexManager(len(cluster_with_base), 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    # 거리 콜백
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return (
            driving_time_matrix[cluster_with_base[from_node], cluster_with_base[to_node]] + duration_time_list[cluster_with_base[to_node]]
        )


    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # TSP 문제 해결
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)

    solution = routing.SolveWithParameters(search_parameters)
    time = solution.ObjectiveValue()
    if solution:
        index = routing.Start(0)
        route = []
        while not routing.IsEnd(index):
            route.append(cluster_with_base[manager.IndexToNode(index)])
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
        return route, time  # 숙소에서 시작하고 종료하는 경로 인덱스 반환
    else:
        return [0], time  # 해결할 수 없는 경우 숙소만 반환
    
# 각 날짜별 군집화된 관광지를 식별하고 TSP 해결
for day in range(total_day):
    # 군집화된 관광지 인덱스 식별
    cluster = [i + 1 for i, cid in enumerate(best_individual) if cid == day]
    if not cluster:
        print(f"Day {day + 1}: No visits planned.")
        continue
    # 해당 군집에 대한 TSP 경로 해결
    tsp_route, total_seconds = solve_tsp_for_cluster(cluster)
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    # 경로 출력
    print(f"Day {day+1} Tour Order(with {hours} hours {minutes} minutes):")
    route_names = [travel_point_name[idx] for idx in tsp_route]
    print(" -> ".join(route_names))
    print()


29817


ValueError: too many values to unpack (expected 2)

In [4]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# 이동 시간 매트릭스 로드
duration_df = pd.read_csv('./tour_distances_matrix.csv', index_col=0)
stay_df = pd.read_csv('./residence_time.csv', index_col=0)
stay_time = stay_df["Avg_Stay_Duration"].values

# 위치 정보 로드
geo_df = pd.read_csv('./geo_info.csv')
geo_df['Cluster'] = KMeans(n_clusters=total_day, random_state=42).fit(geo_df[['Latitude', 'Longitude']]).labels_

# 숙소 정보
base_location = 'Bukchon Hanok Village'

# 각 클러스터별 TSP 문제 해결 및 출력
for day in range(total_day):
    # 현재 클러스터의 관광지 선택
    cluster_locations = [base_location] + geo_df[geo_df['Cluster'] == day]['Name'].tolist()
    # 이동 시간 매트릭스 추출
    cluster_matrix = duration_df.loc[cluster_locations, cluster_locations].to_numpy()

    # TSP 문제 설정 및 해결
    manager = pywrapcp.RoutingIndexManager(len(cluster_matrix), 1, 0)
    routing = pywrapcp.RoutingModel(manager)
    transit_callback_index = routing.RegisterTransitCallback(lambda from_index, to_index: cluster_matrix[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)] + stay_time[to_index])
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    solution = routing.SolveWithParameters(search_parameters)
    # 결과 출력
    total_seconds = solution.ObjectiveValue()
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    print(f'Day {day+1} Tour Order(with {hours} hours {minutes} minutes):')
    index = routing.Start(0)
    while not routing.IsEnd(index):
        print(f'{cluster_locations[manager.IndexToNode(index)]} -> ', end='')
        index = solution.Value(routing.NextVar(index))
    print(base_location)
    print()

Day 1 Tour Order(with 29 hours 31 minutes):
Bukchon Hanok Village -> Changdeokgung Palace -> Myeongdong Cathedral -> Samcheong-dong Alley -> Ikseon-dong Hanok Street -> Gwanghwamun Square -> Deoksugung Palace -> Seoul Plaza -> Bosingak Belfry -> Hwangudan Altar -> Deoksugung Stonewall Walkway -> Bukak Skyway Octagonal Pavilion -> The Blue House -> Gyeongbokgung Palace -> Seodaemun Prison History Museum -> Bukchon Hanok Village -> Bukchon Hanok Village

Day 2 Tour Order(with 8 hours 49 minutes):
Bukchon Hanok Village -> Bangi-dong Baekje Tombs -> Lotte World Tower -> Bukchon Hanok Village

Day 3 Tour Order(with 8 hours 53 minutes):
Bukchon Hanok Village -> 63 Square -> National Assembly Building -> Bukchon Hanok Village

Day 4 Tour Order(with 12 hours 12 minutes):
Bukchon Hanok Village -> National Museum of Korea -> Yongsan Family Park -> Banpo Bridge Night View -> National Library of Korea -> Bukchon Hanok Village

Day 5 Tour Order(with 6 hours 49 minutes):
Bukchon Hanok Village -> Sta